# Exploratory Data Analysis - Univariate Analysis

In [5]:
import json
import pandas as pd
import plotly.express as px

In [3]:
df = pd.read_csv('./../../../data/data.csv')

In [4]:
# Load lists of numerical and categorical columns from the static file
with open('./../../../data/statics.json') as f:
    statics = json.load(f)
categorical_columns = statics['categorical_columns']
numerical_columns = statics['numerical_columns']


NameError: name 'json' is not defined

In [ ]:
# Separate out the dataframe intro numerical and categorical dataframe
num_df = df[numerical_columns]
cat_df = df[categorical_columns]


In [ ]:
# Descriptive statics for numerical variables
num_df.describe()
for column in categorical_columns:
    print(f"Getting category-wise counts for column: {column}")
    print(df[column].value_counts())
    print("\n")